**Estrategia para el modelo predictivo**

*Datos de entrada:* Se utilizaron componentes principales derivados de variables de Saber11, las cuales describen el perfil socioeconómico y académico de los estudiantes. La transformación a componentes principales permite reducir la dimensionalidad del conjunto de datos y retener únicamente la información más relevante para predecir los puntajes en SaberPro.

*Proceso de agrupación de perfiles:* Dado que no existe un vínculo directo entre los resultados de Saber11 y SaberPro para estudiantes individuales, se optó por agrupar los perfiles en base a variables socioeconómicas clave (estrato y educación de los padres). Esta agrupación permite aproximar una relación entre perfiles similares de Saber11 y los puntajes en SaberPro, y brinda al modelo datos representativos de cada perfil para el aprendizaje.

*Modelo predictivo:* Para esta primera aproximación se seleccionó el modelo de regresión lineal, que ofrece una interpretación directa de las relaciones entre variables. La regresión lineal es especialmente útil en este caso, ya que permite analizar cómo cada componente principal (factor generado del perfil socioeconómico y académico) contribuye a los puntajes de SaberPro, proporcionando una visión general y eficiente para el modelado de relaciones lineales.

*Evaluación y ajuste del modelo:* El rendimiento del modelo se evaluó mediante métricas como el error cuadrático medio (MSE) y el coeficiente de determinación (R²), las cuales indican la precisión y capacidad explicativa del modelo. Al observar el desempeño, se busca realizar ajustes que mejoren la capacidad del modelo para capturar patrones relevantes y aumentar la precisión de las predicciones para estudiantes con perfiles socioeconómicos similares a los de Saber11.

**Entrenamiento con Componentes Principales de Perfiles Generales**

1) Se usarán los componentes principales de Saber11 como las características que describen el perfil del estudiante.

2) Los puntajes en SaberPro serán las variables objetivo.

3) El modelo se entrenará con la premisa de que los patrones socioeconómicos y académicos de Saber11 son representativos de los estudiantes en SaberPro.

In [43]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [44]:
# Cargar los datos transformados y listos para el modelo
ruta_saber11 = "Saber11_PCA_components.csv"
ruta_saberpro = "SaberPro_PCA_components.csv"

# Leer los datos de Saber11 y SaberPro
saber11_pca = pd.read_csv(ruta_saber11)
saberpro_pca = pd.read_csv(ruta_saberpro)


In [45]:
# Se va a crear un conjunto de datos reducido de saber11_pca agrupando los perfiles de
# estudiantes según características comunes (estrato y educación de los padres).
# Esto permitirá calcular el promedio de los componentes principales para cada perfil,
# de modo que luego pueda alinearse con los datos de SaberPro.

# Cargar las versiones actualizadas de Saber11 y SaberPro
# ruta_salida = r"C:\Users\jaime\ALUNA Dropbox\JAIME ARTURO RAMIREZ\MIAD-Proyecto"
df_Saber11 = pd.read_csv("Saber11_actualizado.csv")
df_SaberPro = pd.read_csv("SaberPro_actualizado.csv")

In [46]:
# Crear un DataFrame de Saber11 para agrupar y promediar los perfiles
# Combinar componentes principales con variables de perfil
df_saber11_pca = pd.DataFrame(saber11_pca, columns=[f'PC{i+1}' for i in range(saber11_pca.shape[1])])
df_saber11_pca['FAMI_ESTRATOVIVIENDA'] = df_Saber11['FAMI_ESTRATOVIVIENDA']
df_saber11_pca['FAMI_EDUCACIONMADRE'] = df_Saber11['FAMI_EDUCACIONMADRE']
df_saber11_pca['FAMI_EDUCACIONPADRE'] = df_Saber11['FAMI_EDUCACIONPADRE']

# Agrupar por perfil y calcular la media de cada componente principal
df_saber11_grouped = df_saber11_pca.groupby(['FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE']).mean().reset_index()
print("Dimensiones de df_saber11_grouped:", df_saber11_grouped.shape)

Dimensiones de df_saber11_grouped: (343, 11)


In [47]:
# Preparar SaberPro para el emparejamiento

# Crear un DataFrame de SaberPro para usar en el modelo predictivo
df_saberpro_pca = pd.DataFrame(saberpro_pca, columns=[f'PC{i+1}' for i in range(saberpro_pca.shape[1])])
df_saberpro_pca['FAMI_ESTRATOVIVIENDA'] = df_SaberPro['FAMI_ESTRATOVIVIENDA']
df_saberpro_pca['FAMI_EDUCACIONMADRE'] = df_SaberPro['FAMI_EDUCACIONMADRE']
df_saberpro_pca['FAMI_EDUCACIONPADRE'] = df_SaberPro['FAMI_EDUCACIONPADRE']
print("Dimensiones de df_saberpro_pca:", df_saberpro_pca.shape)


Dimensiones de df_saberpro_pca: (1090585, 11)


In [48]:
# Fusión completa para verificar correspondencias exactas
df_combined_full = pd.merge(
    df_saber11_grouped, 
    df_saberpro_pca, 
    on=['FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE'], 
    how='outer', 
    indicator=True, 
    suffixes=('_saber11', '_saberpro')
)

# Ver resumen de coincidencias en la columna '_merge'
print("Resumen de coincidencias en la columna '_merge':")
print(df_combined_full['_merge'].value_counts())

Resumen de coincidencias en la columna '_merge':
_merge
both          1090585
left_only           0
right_only          0
Name: count, dtype: int64


In [49]:
print("Dimensiones de df_combined_full:", df_combined.shape)
print("Resumen de df_combined_full:")
print(df_combined_full.info())

Dimensiones de df_combined_full: (251565, 13)
Resumen de df_combined_full:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090585 entries, 0 to 1090584
Data columns (total 20 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   FAMI_ESTRATOVIVIENDA  1090585 non-null  int64   
 1   FAMI_EDUCACIONMADRE   1090585 non-null  int64   
 2   FAMI_EDUCACIONPADRE   1090585 non-null  int64   
 3   PC1_saber11           1090585 non-null  float64 
 4   PC2_saber11           1090585 non-null  float64 
 5   PC3_saber11           1090585 non-null  float64 
 6   PC4_saber11           1090585 non-null  float64 
 7   PC5_saber11           1090585 non-null  float64 
 8   PC6_saber11           0 non-null        float64 
 9   PC7_saber11           0 non-null        float64 
 10  PC8_saber11           0 non-null        float64 
 11  PC1_saberpro          1090585 non-null  float64 
 12  PC2_saberpro          1090585 non-null  float64 
 1

In [50]:
# Eliminar columnas con solo valores NaN en df_combined
df_combined_full = df_combined_full.dropna(axis=1, how='all')

# Verificar que solo queden las columnas relevantes para el análisis
print("Dimensiones de df_combined después de limpiar columnas NaN:", df_combined_full.shape)
print("Resumen de df_combined después de la limpieza:")
print(df_combined_full.info())


Dimensiones de df_combined después de limpiar columnas NaN: (1090585, 14)
Resumen de df_combined después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090585 entries, 0 to 1090584
Data columns (total 14 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   FAMI_ESTRATOVIVIENDA  1090585 non-null  int64   
 1   FAMI_EDUCACIONMADRE   1090585 non-null  int64   
 2   FAMI_EDUCACIONPADRE   1090585 non-null  int64   
 3   PC1_saber11           1090585 non-null  float64 
 4   PC2_saber11           1090585 non-null  float64 
 5   PC3_saber11           1090585 non-null  float64 
 6   PC4_saber11           1090585 non-null  float64 
 7   PC5_saber11           1090585 non-null  float64 
 8   PC1_saberpro          1090585 non-null  float64 
 9   PC2_saberpro          1090585 non-null  float64 
 10  PC3_saberpro          1090585 non-null  float64 
 11  PC4_saberpro          1090585 non-null  float64 
 12  PC5_s

In [53]:
# Preparación para el Modelo Predictivo

# Seleccionar componentes principales de Saber11 como características (X) y componentes de SaberPro como etiquetas (y)
desired_components_saber11 = 5  # Asegúrate de que coincidan con el PCA realizado en Saber11
desired_components_saberpro = 5  # Asegúrate de que coincidan con el PCA realizado en SaberPro

# Crear las matrices de entrada y salida para el modelo predictivo a partir de df_combined_full
X = df_combined_full[[f'PC{i+1}_saber11' for i in range(desired_components_saber11)]]
y = df_combined_full[[f'PC{i+1}_saberpro' for i in range(desired_components_saberpro)]]

# Verificar que no hay valores NaN
print("\nVerificación de valores NaN en X e y:")
print("Valores NaN en X:", X.isna().sum().sum())
print("Valores NaN en y:", y.isna().sum().sum())

# Mostrar una vista previa de los datos
print("\nDatos de entrada (X):")
print(X.head())
print("\nDatos de salida (y):")
print(y.head())


Verificación de valores NaN en X e y:
Valores NaN en X: 0
Valores NaN en y: 0

Datos de entrada (X):
   PC1_saber11  PC2_saber11  PC3_saber11  PC4_saber11  PC5_saber11
0    -2.921871     0.208384     -0.00292    -0.647181    -0.453373
1    -2.921871     0.208384     -0.00292    -0.647181    -0.453373
2    -2.921871     0.208384     -0.00292    -0.647181    -0.453373
3    -2.921871     0.208384     -0.00292    -0.647181    -0.453373
4    -2.921871     0.208384     -0.00292    -0.647181    -0.453373

Datos de salida (y):
   PC1_saberpro  PC2_saberpro  PC3_saberpro  PC4_saberpro  PC5_saberpro
0     -2.284895      0.620476     -1.992764     -0.852571     -0.170475
1     -3.211992     -0.378507      0.035992     -1.053489      0.150351
2     -3.096207     -1.400784      0.031915     -1.647273      0.354662
3     -2.980096      1.247819     -0.770618     -0.260581     -0.223151
4     -2.609290      1.280324     -1.502275     -0.378262     -0.273269


In [54]:
# Entrenamiento y evaluación el modelo de regresión lineal

# Paso 1: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 2: Entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [55]:
# Paso 3: Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Paso 4: Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


In [56]:
# Imprimir los resultados de la evaluación
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R²):", r2)

# Opcional: Mostrar algunas predicciones junto con los valores reales
predicciones_vs_reales = pd.DataFrame({'Predicciones': y_pred.flatten(), 'Valores Reales': y_test.values.flatten()})
print(predicciones_vs_reales.head())


Error cuadrático medio (MSE): 0.3885657974544093
Coeficiente de determinación (R²): 0.5383791209518025
   Predicciones  Valores Reales
0     -1.181408       -0.882756
1     -0.287166       -0.168289
2     -0.267532       -0.818396
3     -0.042195       -0.090881
4     -0.383780       -0.540346
